### Modelos de otimização

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import default_rng

Os modelos visto até agora não são adequados para representar uma rede que representa uma malha aérea de voos comerciais, em que a estrutura global da rede tem um propósito de tentar minimizar custos das empresas aéreas e tempos de voos dos passageiros. 

Esta última rede é organizada na forma de várias estruturas estrelares em que os centros das estrelas (hubs) são conectados entre si. 

Os centros representam os aeroportos mais movimentados e as folhas das estrelas os aeroportos com menos fluxos de passageiros.

Um modelo de otimização que procure minimizar o custo de operação da malha aérea, que é representado
pelo número de arestas da rede, $m$, e o tempo médio de viagem, que é representado pelo tamanho médio dos
menores caminhos entre os vértices, $\delta$. 

Nesse modelo a proposta é minimizar uma combinação convexa destas duas quantidades, ou seja, minimizar:
$$C = am + (1 − a)\delta$$ 
para $a \in (0,1)$.

O objetivo é dada uma certa quantidade de vértices, $n$, encontrar a rede que minimiza $C$.

Considerando que se houverem vértices que não se comunicam, a distância entre eles seja infinita. 

Então, com o intuito de minimizar $C$ devemos ter uma rede que contém uma única componente.

### Algoritmo de Ferrer i Cancho e Solé

Ferrer i Cancho e Solé, propõem o seguinte algoritmo heurístico de otimização aproximada:

1. Forma-se uma rede aleatória de Erdös-Renyi $G = (V,E)$ com $n$ vértices e com uma densidade inicial $\text{dens}(G)$.

2. Enquanto pelo menos uma das últimas $M$ tentativas de mudança da rede tenha diminuído o valor de $C$
    
    1. Para cada par de vértices $i$, $j$:
        
        1. Com probabilidade $\mu$, decide-se mudar a ligação entre os vértices.
        
            1. Se $(i,j)\in E$, remove-se a aresta.
            2. Se $(i,j)\not\in E$, acrescenta-se a rede com a arestas entre os nós.
            3. Se o valor $C$ da nova rede é menor, se conserva a mudança.
        
Ferrer i Cancho e Solé sugerem usar $m = \binom{n}{2}$, $\mu = 2/m$. 

O algoritmo precisa que a rede seja inicialmente conectada, mas pode-se adaptar como mostra o script a seguir. 

Se a rede for inicialmente não conectada, novas arestas serão acrescentadas.

In [ ]:
rng = default_rng()

def Energia(a,g):
    if nx.is_connected(g) == False:
        return np.inf
    else:
        return a*len(g.edges) + (1-a)*nx.average_shortest_path_length(g)

In [ ]:
n = 50
m = n*(n-1)/2
mu = 2/m
rho = 0.2

In [ ]:
G = nx.gnp_random_graph(n,rho)

In [ ]:
# rede original
nx.draw(G, with_labels=True)

In [ ]:
# teste a = 0, 0.1, 0.5, 1
a = 0.1
semmud = 0
Enat = Energia(a,G)
k = 0

while semmud < m:
    for i in range(n):
        for j in range(i+1,n):
            if rng.random() > mu:
                continue
            if (i,j) in G.edges:
                G.remove_edge(i,j)
                ene = Energia(a,G)
                if ene < Enat:
                    Enat = ene
                    semmud = 0
                else:
                    semmud += 1
                    G.add_edge(i,j)
            else:
                G.add_edge(i,j)
                ene = Energia(a,G)
                if ene < Enat or Enat == np.inf:
                    Enat = ene
                    semmud = 0
                else:
                    semmud += 1
                    G.remove_edge(i,j)
            k += 1
            if k%1000 == 0:
                print(k,Enat,semmud)
            if semmud >= m:
                break
        if semmud >= m:
            break

In [ ]:
# rede otimizada
nx.draw(G, with_labels=True)